In [ ]:
# === colab configuration ===
# p.s. when training on colab, weights are saved on Drive (directory DSCAS/weights).
# p.p.s. skip this cell if running demo file locally!

! pip install torch-summary
! pip install ray
from google.colab import drive
import sys

# setting paths
repository_path = '/content/deep-seasonal-color-analysis-system/'
dataset_path = repository_path + 'headsegmentation_dataset_ccncsa/'
dataset_path_drive = '/content/drive/MyDrive/DSCAS/headsegmentation_dataset_ccncsa/'
weights_path = repository_path + 'models/weights/'
weights_path_drive = '/content/drive/MyDrive/DSCAS/weights/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/mrcmich/deep-seasonal-color-analysis-system.git
! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main__fastscnn_hyperparameters_tuning'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from models import dataset, training_and_testing
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from models.config import *
from PIL import Image
import torchvision.transforms as T


class MyDataset2(Dataset):
    def __init__(self, img_paths):
        self.img_paths = img_paths
        self.pil_to_tensor = T.Compose([T.PILToTensor()])
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index]).convert('RGB')
        image = self.pil_to_tensor(image)
        return image.float() / 255


dataset_path = ROOT_DIR + 'headsegmentation_dataset_ccncsa/'
# tH, tW = 272, 288  # image sizes must be multiple of 16

# fetching dataset
img_paths, label_paths = dataset.get_paths(dataset_path, file_name='training.xml')
X_train, X_test, Y_train, Y_test = train_test_split(img_paths, label_paths, test_size=0.20, random_state=99, shuffle=True)
train_dataset = MyDataset2(X_train)

# data loader
image_loader = DataLoader(train_dataset, 
                          batch_size=1, 
                          drop_last=False)

psum = torch.zeros((3), dtype=torch.float32)
psum_sq = torch.zeros((3), dtype=torch.float32)
count = 0

for image in tqdm(image_loader):
    psum += image.sum(dim=[0, 2, 3])
    psum_sq += (image ** 2).sum(dim=[0, 2, 3])
    count += image.shape[2] * image.shape[3]  # pixel count

mean = psum / count
var  = (psum_sq / count) - (mean ** 2)
std  = torch.sqrt(var)
print(f'mean: {mean}')
print(f'var: {var}')
print(f'std: {std}')